<a href="https://colab.research.google.com/github/leon123858/YOLO_exercise/blob/master/Mask_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#引入一個開源運算框架 
! git clone https://github.com/pjreddie/darknet.git
# 修改make file
! head darknet/Makefile
!sed -i "s/GPU=0/GPU=1/g" darknet/Makefile
!sed -i "s/CUDNN=0/CUDNN=1/g" darknet/Makefile
!sed -i "s/OPENCV=0/OPENCV=1/g" darknet/Makefile
#編譯
! cd darknet; make

In [ ]:
#抓取訓練資料
!wget https://tinyurl.com/y6d6p24o

In [6]:
#更新成可用的檔名
!mv /content/y6d6p24o /content/medical-masksdataset.zip

mv: cannot stat '/content/y6d6p24o': No such file or directory


In [8]:
#解壓縮
! unzip /content/medical-masksdataset.zip > data_unzip.log

In [ ]:
#查看圖片對應的XML 用支援YOLO的標註器完成
! cat /content/labels/012106.jpg_1140x855.xml

In [11]:
#建立資料夾
! rm -rf /content/yolo
! mkdir /content/yolo

In [ ]:
from bs4 import BeautifulSoup
import os
import shutil

status_dio = {'good':2,'bad':1,'none':0}
def getYoloFormat(filename,label_name,img_path,yolo_path,newname):
  with open(label_)